In [1]:
import pandas as pd
from datetime import datetime
from gsheets_connect import GoogleSheets, Connect
import helper_functions as hlp
from expense import Expense
import os
from credit_card_object import CreditCard
from log_expense_gui import LogExpenseForm, AvailableFilesForm

GSHEET_ID = '14a36tesQZ2AH0aIEdG5Ch2F8iVyrnXCvlxeASLX47N4'

GSHEET = GoogleSheets()

mnths_dict = {
    'Ene':'01','Feb':'02','Mar':'03',
    'Abr':'04','May':'05','Jun':'06',
    'Jul':'07','Ago':'08','Sep':'09',
    'Oct':'10','Nov':'11','Dic':'12'
}

# Helper Functions

In [2]:
def loadXlsx(file_path):
    data = pd.read_excel(file_path, header=None)
    
    ix_of_first = index_of_first_spend(data)
    lst_4_digits = try_finding_card_num(data)
    
    data = data.iloc[
        ix_of_first:,:3
    ].copy().reset_index(drop=True)
    
    # Date column as datetime
    data.iloc[:,0] = data.iloc[:,0].apply(
        lambda x: parse_date(x)[1]
    )
    
    # Amount column as float
    data.iloc[:,2] = data.iloc[:,2].apply(
        lambda x: parse_amount(x)
    )
    
    return data, lst_4_digits

def loadXls(file_path):
    file = pd.read_html(file_path)
    data = file[1]
    
    ix_of_first = index_of_first_spend(data)
    lst_4_digits = try_finding_card_num(data)
    
    data = data.iloc[
        ix_of_first:,[0,2,3]
    ].copy().reset_index(drop=True)
    
    # Date columns as datetime
    data.iloc[:,0] = data.iloc[:,0].apply(
        lambda x: parse_date(x)[1]
    )
    
    # Amount column as float
    data.iloc[:,2] = data.iloc[:,2].apply(
        lambda x: parse_amount(x)
    )
    
    return data, lst_4_digits

def get_available_files():
    data_sources = []
    cwd = os.getcwd()
    files_path = os.path.join(cwd,"input_data")
    dir_content = os.listdir(files_path)
    for file_name in dir_content:
        file_path = os.path.join(files_path, file_name)
        if file_path.endswith('.xlsx') or file_path.endswith('.xls'):
            data_sources.append(file_path)

    return data_sources

def open_file(file_path):
    if file_path.endswith('.xlsx'):
        loaded_data, lst_4_digits = loadXlsx(file_path)
    elif file_path.endswith('.xls'):
        loaded_data, lst_4_digits = loadXls(file_path)
    else:
        raise ValueError("Unable to open file: {0}".format(file_path))
    return loaded_data, lst_4_digits

def load_file(file_to_process):
    
    data, lst_4_digits = open_file(file_to_process)
    
    #print("Data loaded correctly \n")
    return data.dropna(), lst_4_digits

def index_of_first_spend(data_frame):
    for ix, row in data_frame.iterrows():
        val = row.iloc[0]
        if not pd.isnull(val) and parse_date(val)[0]:
            return ix
    return None

def try_finding_card_num(data_frame):
    
    for ix, row in data_frame.iterrows():
        val = row.iloc[0]
        if (
            'Adicional' in str(val) or 
            'Titular' in str(val) or 
            'No. de Cuenta' in str(val) or
            'Digital' in str(val)
        ):
            striped_str = ''.join(e for e in val if e.isalnum())
            # Will contain strings that are sequential numbers
            container = []
            # Will contain sequential numbers (not separated by characters)
            a_possible_num=[]
            for i, s in enumerate(striped_str):
                if is_num(s):
                    # keep appending this number
                    a_possible_num.append(s)
                    if i == len(striped_str)-1:
                        # its the end of the string
                        possible_num_str = ''.join(a_possible_num)
                        container.append(possible_num_str)
                else:
                    # non numeric character found in the string
                    possible_num_str = ''.join(a_possible_num)
                    container.append(possible_num_str)
                    a_possible_num=[]
            return [c for c in container if len(c)>=4][0][-4:]
    return '0000'

def parse_date(date_string):
    try:
        date_string = parse_month(date_string)
        date = datetime.strptime(date_string, '%d/%m/%Y')
        return (True, date)
    except (ValueError, TypeError):
        return (False, None)
    
def parse_amount(num_str):
    try:
        num = float(num_str)
    except ValueError:
        num = float(num_str.replace(',', ''))
    return num

def parse_month(date_string):
    for m,v in mnths_dict.items():
        if m in date_string:
            date_string = date_string.replace(m,v)
    return date_string

def update_expenses_data_frame(cur_df, expense, ignored=False):
    
    lst_ix = 0 if cur_df.shape[0] == 0 else cur_df.index.max()+1
    new_df = pd.DataFrame(index=[lst_ix], columns=cur_df.columns)
    
    new_df.iloc[0].loc["expense_id"] = expense.getId()
    new_df.iloc[0].loc["expense_date"]=expense.getDateAsString()
    new_df.iloc[0].loc["description"] = expense.getDescription()
    new_df.iloc[0].loc["amount"]=expense.getAmount()
    new_df.iloc[0].loc["installments"]=expense.getInstallments()
    new_df.iloc[0].loc["ignored"]=ignored
    new_df.iloc[0].loc["payment_method"]=expense.getPaymentMethod()
    new_df.iloc[0].loc["method_name"]=expense.getPaymentMethodName()
    
    cur_df = pd.concat([cur_df, new_df])
    
    return cur_df

def update_installments_data_frame(cur_df, expenses):
    
    for i, expense in enumerate(expenses):
        lst_ix = 0 if cur_df.shape[0] == 0 else cur_df.index.max()+1
        new_df = pd.DataFrame(index=[lst_ix], columns=cur_df.columns)
        
        new_df.iloc[0].loc["expense_id"] = expense.getId()
        new_df.iloc[0].loc["installment_amount"]=expense.getAmount()
        new_df.iloc[0].loc["expense_date"] = expense.getDateAsString()
        new_df.iloc[0].loc["expense_month"]=expense.getMonthNum()
        new_df.iloc[0].loc["payment_date"]=expense.getPaymentDateAsString()
        new_df.iloc[0].loc["payment_month"]=expense.getPaymentMonthNum()
        new_df.iloc[0].loc["total_installments"]=expense.getInstallments()
        new_df.iloc[0].loc["installment_num"]=str(i)
        new_df.iloc[0].loc["payment_method"]=expense.getPaymentMethod()
        new_df.iloc[0].loc["method_name"]=expense.getPaymentMethodName()
        new_df.iloc[0].loc["category"]=expense.getMainCategory()
        new_df.iloc[0].loc["sub_category"]=expense.getSubCategory()
        new_df.iloc[0].loc["payment_fortnight"]=expense.getPaymentFortnight()
        new_df.iloc[0].loc["description"]=expense.getDescription()
        
        cur_df = pd.concat([cur_df, new_df])
    
    return cur_df
            
def updateData(
    an_expense, 
    row, 
    category, 
    sub_category, 
    installments, 
    pay_method_type, 
    credit_card_used
):
    
    date, concept, amount = row
    
    an_expense.updateData(
        payment_date=date, 
        description=concept, 
        category=category, 
        sub_category=sub_category, 
        amount=amount, 
        payment_method=pay_method_type, 
        n_installments=installments,
        credit_card_used=credit_card_used
    )
    
def pivotData(data_frame, how='bimonthly'):
    
    date_ix = 'payment_'+('fortnight' if how =='bimonthly' else 'month')
    pivot = pd.pivot_table(
        data_frame, 
        values = 'installment_amount',
        columns='method_name',
        aggfunc='sum',
        index=[date_ix, 'category', 'sub_category']
    ).fillna(0)
    pivot['Total'] = pivot.sum(axis=1)

    pivot2 = pivot.copy()
    pivot2.drop('Total', axis=1, inplace=True)

    to_be_divided = ['Pixies', 'Super']
    for c in pivot2.columns:
        if c in to_be_divided:
            pivot2[c] /= 2

    pivot['MyTotal'] = pivot2.sum(axis=1)

    pivot.reset_index(inplace=True)
    
    return pivot

def list_payment_methods(credit_cards_list):
    payment_methods = []
    for cc in credit_cards_list:
        payment_methods.append(cc.alias_name)
    payment_methods.append('debit')
    payment_methods.append('cash')
    return payment_methods

def load_files_form(available_files):
    file_names = [f.split('/')[-1] for f in available_files]
    dialog = AvailableFilesForm(
        available_files = file_names
        #available_methods = payment_methods
    )
    result = -1
    if dialog.exec_() == AvailableFilesForm.Accepted:
        result = dialog.get_output()
    
    if result == -1:
        raise RuntimeError("Program Terminated")
    
    file_ix = result
    file_to_open = available_files[file_ix]
    
    return file_to_open

def is_num(char):
    """
    Receives a character and returns if the character is a number.
    Ex.
        'a' -> False
        '22' -> True
        '34.5' -> True
        '?' -> Flase
    """
    try:
        float(char)
        return True
    except ValueError:
        return False

# Load Data

In [3]:
def load_data_from_files():
    ccs = hlp.load_credit_cards()
    ccs_lst = list_payment_methods(ccs)

    data_dict = {}
    for i, file_to_open in enumerate(get_available_files()):
        dd = {}
        data, card_last_4_digits = load_file(file_to_open)
        dd['lst_4_digits'] = card_last_4_digits

        PAYMENT_METHOD_TYPE = 'credit' if card_last_4_digits in [cc.get_last_four_digits() for cc in ccs] else 'debit'
        CREDIT_CARD_USED = ccs[
            [cc.get_last_four_digits() for cc in ccs].index(card_last_4_digits)
        ] if PAYMENT_METHOD_TYPE == 'credit' else None

        if PAYMENT_METHOD_TYPE=='debit':
            data.iloc[:,2] = data.iloc[:,2]*-1

        dd['data'] = data
        dd['pay_method'] = PAYMENT_METHOD_TYPE
        dd['credit_card_object'] = CREDIT_CARD_USED

        data_dict[i] = dd
    print('Data Loaded Correctly')
    return data_dict

## Load current Data

In [4]:
def load_current_data_from_gsheet():
    global LAST_ID
    global EMPTY_DATA
    try:
        cur_inst_data = hlp.retrieveDataFromSheet(
            GSHEET, 
            GSHEET_ID, 
            sheet_name = 'data'
        )

        cur_inst_data['expense_id'] = cur_inst_data['expense_id'].astype(int)
        cur_inst_data['installment_amount'] = cur_inst_data['installment_amount'].str.replace(',', '').astype(float)
        cur_inst_data['expense_month'] = cur_inst_data['expense_month'].astype(int)
        cur_inst_data['payment_month'] = cur_inst_data['payment_month'].astype(int)
        cur_inst_data['installment_num'] = cur_inst_data['installment_num'].astype(str)
        cur_inst_data['total_installments'] = cur_inst_data['total_installments'].astype(int)

        cur_exp_data = hlp.retrieveDataFromSheet(
            GSHEET, 
            GSHEET_ID, 
            sheet_name = 'expenses'
        )

        cur_exp_data['expense_id'] = cur_exp_data['expense_id'].astype(int)
        cur_exp_data['amount'] = cur_exp_data['amount'].str.replace(',', '').astype(float)
        cur_exp_data['ignored'] = cur_exp_data['ignored'].astype(str)
        cur_exp_data['installments'] = cur_exp_data['installments'].astype(int)

        cur_exp_data['key'] = (
            cur_exp_data['expense_date'] +'_'+
            cur_exp_data['description']+'_'+
            round(cur_exp_data['amount'].astype(float)).astype(int).astype(str)
        )

        s1 = set(cur_exp_data['key'])
        LAST_ID = cur_exp_data['expense_id'].max()
        
        cur_exp_data.drop('key',axis=1, inplace=True)

        EMPTY_DATA=False

    except IndexError:
        EMPTY_DATA = True
        LAST_ID = 0
        s1 = set()
    
    return cur_inst_data, cur_exp_data, s1

In [5]:
def get_missing_data(data, set_of_cur_data):
    positive_data = data[data.iloc[:,2].astype(float)>0].copy()
    positive_data['key'] = (
        positive_data.iloc[:,0].astype(str)+'_'+
        positive_data.iloc[:,1]+'_'+
        round(positive_data.iloc[:,2].astype(float)).astype(int).astype(str)
    )

    s2 = set(positive_data['key'])
    
    missing_data = positive_data[
        ~positive_data['key'].isin(
            set_of_cur_data.intersection(s2)
        )
    ].reset_index(drop=True).drop('key', axis=1).copy()
    return missing_data

In [6]:
def init_dataframes():
    # Init Data Frames
    cols_1 = [
        "expense_id",
        "installment_amount",
        "expense_date", 
        "expense_month", 
        "payment_date", 
        "payment_month", 
        "total_installments", 
        "installment_num",
        "payment_method",
        "method_name",
        "category", 
        "sub_category",
        "payment_fortnight",
        "description"
    ]

    cols_2 = [
        "expense_id",
        "expense_date", 
        "description", 
        "amount",
        "installments",
        "ignored",
        "payment_method",
        "method_name"
    ]

    installments_df = pd.DataFrame(index=[0],columns=cols_1)
    expenses_df = pd.DataFrame(index=[0],columns=cols_2)
    return installments_df, expenses_df

# Log Expenses

In [7]:
def log_expenses(missing_data, payment_method_type, cc_used):
    print('logging expenses...')
    new_expenses = []
    main_cats, sub_cats = hlp.getCategoriesFromGSheet(GSHEET, GSHEET_ID)
    
    installments_df, expenses_df = init_dataframes()

    for ix, row in missing_data.iterrows():
        an_expense = Expense(this_id=(int(LAST_ID)+1)+ix)
        dialog = LogExpenseForm(
            categories = main_cats, 
            subcategories = sub_cats, 
            expense_date_str = str(row.iloc[0].date()),
            expense_description = row.iloc[1],
            expense_amount = str(row.iloc[2]),
            payment_type=payment_method_type,
            card_last_4_digits = cc_used.get_last_four_digits()
        )

        log_expense = None
        if dialog.exec_() == LogExpenseForm.Accepted:
            log_expense, cat, sub_cat, installments = dialog.get_output()

        if log_expense is None:
            break
        elif log_expense == 1:

            updateData(
                an_expense=an_expense, 
                row=row, 
                category=cat, 
                sub_category=sub_cat, 
                installments=installments,
                pay_method_type=payment_method_type, 
                credit_card_used=cc_used
            )
            sub_expenses = an_expense.divideExpense()
            installments_df = update_installments_data_frame(
                installments_df, 
                expenses = sub_expenses
            )
            expenses_df = update_expenses_data_frame(
                cur_df = expenses_df, 
                expense = an_expense, 
                ignored=False
            )
        else:
            updateData(
                an_expense=an_expense, 
                row=row, 
                category=cat, 
                sub_category=sub_cat, 
                installments=1,
                pay_method_type=payment_method_type, 
                credit_card_used=cc_used
            )
            expenses_df = update_expenses_data_frame(
                cur_df = expenses_df, 
                expense = an_expense, 
                ignored=True
            )

    return installments_df, expenses_df

In [8]:
def update_data_on_gsheet(cur_inst_data, cur_exp_data, new_installments_data, new_expenses_data):
    
    updated_inst_data = pd.concat(
        [cur_inst_data, new_installments_data], 
        ignore_index=True
    ) if not EMPTY_DATA else new_installments_data
    updated_inst_data.sort_values(by=['expense_date'], inplace=True)
    updated_inst_data.index = range(len(updated_inst_data))
    updated_inst_data.drop_duplicates(
        subset=['expense_date', 'description', 'installment_amount'],
        inplace=True
    )

    inst_values_list = hlp.dataFrameToListOfValues(updated_inst_data.dropna())

    GSHEET.clear_values(
        spreadsheet_id = GSHEET_ID,
        range_name = 'data'
    )

    GSHEET.values_to_gsheet(
        spreadsheet_id = GSHEET_ID,
        values_list=inst_values_list, 
        range_name='data'
    )
    
    updated_exp_data = pd.concat(
        [cur_exp_data, new_expenses_data], 
        ignore_index=True 
    )if not EMPTY_DATA else new_expenses_data
    updated_exp_data.sort_values(by=['expense_id'], inplace=True)
    updated_exp_data.index = range(len(updated_exp_data))
    updated_exp_data.drop_duplicates(
        subset=['expense_date', 'description', 'amount'],
        inplace=True
    )

    exp_values_list = hlp.dataFrameToListOfValues(updated_exp_data.dropna())

    GSHEET.clear_values(
        spreadsheet_id = GSHEET_ID,
        range_name = 'expenses'
    )

    GSHEET.values_to_gsheet(
        spreadsheet_id = GSHEET_ID,
        values_list=exp_values_list, 
        range_name='expenses'
    )
    
    # Pivot bimonthly (every 15 days)
    bimonthly_pivot = pivotData(updated_inst_data, how='bimonthly')
    bimonthly_values = hlp.dataFrameToListOfValues(bimonthly_pivot)
    GSHEET.clear_values(
        spreadsheet_id = GSHEET_ID,
        range_name = 'bimonthly pivot'
    )
    GSHEET.values_to_gsheet(
        spreadsheet_id = GSHEET_ID,
        values_list=bimonthly_values, 
        range_name='bimonthly pivot!A1'
    )

    # Pivot monthly
    monthly_pivot = pivotData(updated_inst_data, how='monthly')
    monthly_values = hlp.dataFrameToListOfValues(monthly_pivot)
    GSHEET.clear_values(
        spreadsheet_id = GSHEET_ID,
        range_name = 'monthly pivot'
    )
    GSHEET.values_to_gsheet(
        spreadsheet_id = GSHEET_ID,
        values_list=monthly_values, 
        range_name='monthly pivot!A1'
    )
    
    return updated_inst_data, updated_exp_data, bimonthly_pivot, monthly_pivot

In [10]:
data_sets_info

{0: {'lst_4_digits': '4703',
  'data':            0                        1        2
  0 2020-07-17          SUPERAMA DAKOTA  1675.96
  1 2020-07-12    AMAZON MX MARKETPLACE  1093.11
  3 2020-07-11    AMAZON MX MARKETPLACE   380.36
  4 2020-07-08      D LOCAL*CRUNCHYROLL    99.00
  5 2020-07-06  UBER EATS HELP.UBER.COM   201.00
  6 2020-07-02        PAYPAL *ADOBE INC   241.91,
  'pay_method': 'credit',
  'credit_card_object': <credit_card_object.CreditCard at 0x1204f1650>},
 1: {'lst_4_digits': '8188',
  'data':             0                       2         3
  0  2020-07-23   AMAZON MX MARKETPLACE    426.80
  1  2020-07-23   AMAZON MX MARKETPLACE    153.53
  2  2020-07-23               AMAZON MX    264.57
  3  2020-07-21               AMAZON MX    565.27
  4  2020-07-21   AMAZON MX MARKETPLACE   2356.00
  5  2020-07-20        TELCEL MEXICO CR    348.99
  6  2020-07-17   AMAZON MX MARKETPLACE   -850.00
  7  2020-07-16   AMAZON MX MARKETPLACE    850.00
  8  2020-07-16   AMAZON MX MARKE

In [9]:
# Run whole process
data_sets_info = load_data_from_files()

for ix, val in data_sets_info.items():
    data = val['data']
    
    cur_inst_data, cur_exp_data, s1 = load_current_data_from_gsheet()
    data_to_upload = get_missing_data(data=data, set_of_cur_data=s1)
    installments_df, expenses_df = log_expenses(
        missing_data=data_to_upload, 
        payment_method_type = val['pay_method'],
        cc_used = val['credit_card_object']
    )
    updated_inst_data, updated_exp_data, bimonthly_pivot, monthly_pivot = update_data_on_gsheet(
        cur_inst_data = cur_inst_data, 
        cur_exp_data = cur_exp_data, 
        new_installments_data = installments_df, 
        new_expenses_data = expenses_df
    )

Data Loaded Correctly
logging expenses...
7336 cells updated.
4736 cells updated.
1870 cells updated.
1342 cells updated.
logging expenses...
7518 cells updated.
4856 cells updated.
1947 cells updated.
1397 cells updated.
logging expenses...
7546 cells updated.
4872 cells updated.
1958 cells updated.
1397 cells updated.
logging expenses...
7672 cells updated.
4952 cells updated.
1969 cells updated.
1397 cells updated.
logging expenses...
7686 cells updated.
4976 cells updated.
1969 cells updated.
1397 cells updated.
logging expenses...
7686 cells updated.
4976 cells updated.
1969 cells updated.
1397 cells updated.
logging expenses...
7686 cells updated.
4976 cells updated.
1969 cells updated.
1397 cells updated.
logging expenses...
7742 cells updated.
5016 cells updated.
1980 cells updated.
1408 cells updated.
logging expenses...
7798 cells updated.
5048 cells updated.
2002 cells updated.
1408 cells updated.
logging expenses...
7798 cells updated.
5048 cells updated.
2002 cells updated

AttributeError: 'NoneType' object has no attribute 'get_last_four_digits'

In [ ]:
raise Exception

# Manually Analyze expenses

In [13]:
updated_inst_data[
    (updated_inst_data['payment_fortnight'] == '2020-02-29')&
    (updated_inst_data['method_name']=='Bancomer-Platinum')
][['installment_amount', 'expense_date', 'payment_fortnight', 'description']]

,installment_amount,expense_date,payment_fortnight,description
209,495,2020-01-23,2020-02-29,01 DE 03 SUPERAMA DAKOTA
210,626,2020-01-23,2020-02-29,01 DE 06 CAMZNMARKETPLACEMXINS
211,357,2020-01-23,2020-02-29,01 DE 03 SUPERAMA DAKOTA
212,621.36,2020-01-23,2020-02-29,06 DE 06 CAMZNRETAILMXINSTALLM
213,400,2020-01-23,2020-02-29,04 DE 06 EL PALACIO HIERRO POL
214,1159,2020-01-23,2020-02-29,03 DE 03 OPENPAY*COLCHONES ONL
215,218.85,2020-01-23,2020-02-29,AMAZON MX
217,62.96,2020-01-24,2020-02-29,UBER TRIP HELP.UBER.COM
218,356,2020-01-25,2020-02-29,REST FLASH TACO
219,151,2020-01-26,2020-02-29,TOSTADAS AMATISTA


In [49]:
updated_inst_data.columns

Index(['expense_id', 'installment_amount', 'expense_date', 'expense_month',
       'payment_date', 'payment_month', 'total_installments',
       'installment_num', 'payment_method', 'method_name', 'category',
       'sub_category', 'payment_fortnight', 'description'],
      dtype='object')

In [50]:
updated_inst_data['sub_category'].unique()

array(['Other', 'Books', 'Super Market', 'Restaurants', 'Vitamins',
       'Take-out and bingeing', 'Personal Care', 'Hobbies',
       'Stuff for me', 'Clothing', 'Gifts', 'Stuff for the Apartment',
       'Medicine', 'Gas', 'Supplements', 'Coffe', 'Haircut',
       'Socializing and Bars', 'Services', nan], dtype=object)

In [75]:
updated_inst_data[
    (pd.to_datetime(updated_inst_data['expense_date']).dt.month==2)&
    (pd.to_datetime(updated_inst_data['expense_date']).dt.year==2020)&
    #(updated_inst_data['method_name']=='Bancomer-Platinum')&
    (updated_inst_data['sub_category']=='Super Market')
    
][['installment_amount', 'expense_date', 'description']]['installment_amount'].sum() - 1736

3868.9400000000005

In [68]:
updated_inst_data[
    #(pd.to_datetime(updated_inst_data['expense_date']).dt.month==2)&
    #(pd.to_datetime(updated_inst_data['expense_date']).dt.year==2020)&
    #(updated_inst_data['method_name']=='Bancomer-Platinum')&
    #(updated_inst_data['sub_category']=='Super Market')&
    (updated_inst_data['description'].str.contains('CITY MARKET'))&
    (updated_inst_data['installment_amount']>2000)
    
]

,expense_id,installment_amount,expense_date,expense_month,payment_date,payment_month,total_installments,installment_num,payment_method,method_name,category,sub_category,payment_fortnight,description
289,338,2141.09,2020-02-22,202002,2020-02-29,202002,1,0,credit,Bancomer-Platinum,Food,Restaurants,2020-02-29,CITY MARKET PILARES


In [15]:
updated_exp_data[
    (~updated_exp_data['description'].isnull())&
    (updated_exp_data['description'].str.contains('GALANGA'))
]

,expense_id,expense_date,description,amount,installments,ignored,payment_method,method_name
299,300,2020-02-19,REST GALANGA MONTERREY,1485,1,FALSE,credit,Bancomer-Platinum
308,309,2019-12-19,REST GALANGA MONTERREY,1284,1,FALSE,credit,Bancomer-Platinum


In [16]:
updated_inst_data[updated_inst_data['expense_id']==381]

,expense_id,installment_amount,expense_date,expense_month,payment_date,payment_month,total_installments,installment_num,payment_method,method_name,category,sub_category,payment_fortnight,description


In [17]:
updated_exp_data[updated_exp_data['method_name']=='Bancomer-Platinum'].sort_values(by='expense_date', ascending==False)

SyntaxError: positional argument follows keyword argument (<ipython-input-17-fb56c2261450>, line 1)

In [ ]:
installments_df

In [ ]:
updated_inst_data

In [ ]:
updated_exp_data.drop(updated_exp_data[
    (~updated_exp_data['expense_id'].isin(updated_inst_data['expense_id'].unique()))&
    ((updated_exp_data['ignored']==False) | (updated_exp_data['ignored']=='FALSE'))
].index, inplace=True)

In [ ]:
updated_exp_data.sort_values(by=['expense_id'], inplace=True)
updated_exp_data.index = range(len(updated_exp_data))
updated_exp_data.drop_duplicates(
    subset=['expense_date', 'description', 'amount'],
    inplace=True
)

exp_values_list = hlp.dataFrameToListOfValues(updated_exp_data.dropna())

gsheet.clear_values(
    spreadsheet_id = GSHEET_ID,
    range_name = 'expenses'
)

gsheet.values_to_gsheet(
    spreadsheet_id = GSHEET_ID,
    values_list=exp_values_list, 
    range_name='expenses'
)